In [ ]:
"""
This script should code for analysis of data post peak finding to figure generation

Input:
1. metabolite database in xml (HMDB)
2. m/z count table in csv; columns are: m/z@ret_time, m/z, retention time, AUC for each location (both + and - mode)
3. a csv file describing how to read the m/z count table columns

Process:
1. compile all results into one massive table
    table contains column of: m/z@ret_time, m/z, ret_time, AUC, mode, sample_location, sample_strategy [Grab or Composite], 
    sample_timing [Morning, Afternoon, Evening, or NA], sample_catchment_size [1-1000: Hospitals, 1000-10000: Castellar, >10000: Ciscar],
    replicate [1,2,3]
2. for the 2 sampling strategies, do preliminary analysis based on mz@ret_time pattern between different locations
    1a. stacked barplot, x axis is different samples, y axis is N of unique m/z@ret_time, and color is negative and positive
    1b. boxplot, x axis is different samples, y axis is auc
    1c. PCA plots: 1) all data, delineate sampling strategy; 2) for each sampling strategy, delineate location and catchment size; 
        3) for grab samples, delineate time 
    1d. Fold change analysis: for composite, differentiate between hospitals and non-hospitals; for grab, differentiate between different
        times of the day.
3. convert metabolite database to a dictionary
    1a. read the xml file via iteration strategy
    1b. create dictionary of {metabolite_ID: {name:..., neutral_mass:..., taxonomy_info:..., ontology_info:...}, ...}
4. match the found m/z with the one in dictionary to identify what it could potentially be. 

"""

In [1]:
import pandas as pd
import re
import os
import json
import numpy as np
import itertools
import math
from math import pi

from lxml import etree as ET

import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap
from matplotlib import cm, colors
from adjustText import adjust_text

from scipy import stats as st
from sklearn.cluster import OPTICS
from sklearn.manifold import MDS

In [3]:
#read excel file containing the metabolite abundance and making a table the columns below
df = pd.DataFrame(columns=['Compound', 'Mass', 'Retention Time', 'Ion Species', 'lcms_mode','sampling_strategy', 'Info', 'auc'])
count = 0
for ll in os.listdir("raw"):
    if ll.endswith(".xlsx"):
        for oo in [0, 1]:
            directory = "raw/"+ll
            tmpdf = pd.read_excel(directory, sheet_name = oo).melt(id_vars=['Compound', 'Mass', 'Retention Time', 'Ion Species', 'lcms_mode','sampling_strategy'], var_name = 'Info', value_name = 'auc')
            tmpdf['sampling_round'] = count
            df = pd.concat([df, tmpdf])
        count +=1
            
df['TIC'] = df.groupby('Info')['auc'].transform('sum')
df['norm_auc'] = df['auc'] / df['TIC']
            
ont = df['Info'].str.split(pat="-", expand = True)

df['Location'] = ont[0]
df['Time'] = ont[1]
df['Replicate'] = ont[2]

#anonymizing the locations
anonloc_dict = {
    'PESET' : 'Hospital A',
    'GENERAL' : 'Hospital B',
    'U.CLINICO': 'Hospital D',
    'POLITECNICO': 'Hospital F',
    'CISCAR': 'WW Collection Point City Center',
    'MALVA': 'Hospital E',
    'CASTELLAR': 'WW Collection Point Outskirts',
    'CIPS': 'Clinic Sexual Health',
}

df['Anonymized Name'] = df['Location'].map(anonloc_dict)
df['Anonymized Sample ID'] =  df['Anonymized Name']+"-"+df['Time']+"-"+df['Replicate']

df

,Compound,Mass,Retention Time,Ion Species,lcms_mode,sampling_strategy,Info,auc,sampling_round,TIC,norm_auc,Location,Time,Replicate,Anonymized Name,Anonymized Sample ID
0,298.1691@18.483,298.1691,18.483000,[M-H]-,Negative,Composite,PESET-NA-1,74843648.0,0.0,3.225706e+09,2.320225e-02,PESET,NA,1,Hospital A,Hospital A-NA-1
1,200.0562@8.356001,200.0562,8.356001,[M-H]-,Negative,Composite,PESET-NA-1,94551728.0,0.0,3.225706e+09,2.931195e-02,PESET,NA,1,Hospital A,Hospital A-NA-1
2,298.1644@18.550999,298.1644,18.550999,[M-H]-,Negative,Composite,PESET-NA-1,62121756.0,0.0,3.225706e+09,1.925834e-02,PESET,NA,1,Hospital A,Hospital A-NA-1
3,274.1812@10.571,274.1812,10.571000,[2M-H]- [M+Cl]+ [M-H]-,Negative,Composite,PESET-NA-1,58800176.0,0.0,3.225706e+09,1.822862e-02,PESET,NA,1,Hospital A,Hospital A-NA-1
4,790.8776@4.8180003,790.8776,4.818000,[M+Cl]+ [M-H]-,Negative,Composite,PESET-NA-1,1582701.0,0.0,3.225706e+09,4.906526e-04,PESET,NA,1,Hospital A,Hospital A-NA-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42223,249.0695@5.317,249.0695,5.317000,[M-H]-,Negative,Grab,PESET-Evening-3,1.0,1.0,8.795419e+09,1.136955e-10,PESET,Evening,3,Hospital A,Hospital A-Evening-3
42224,374.126@6.756,374.1260,6.756000,[M-H]-,Negative,Grab,PESET-Evening-3,1.0,1.0,8.795419e+09,1.136955e-10,PESET,Evening,3,Hospital A,Hospital A-Evening-3
42225,270.2218@16.536,270.2218,16.536000,[M-H]-,Negative,Grab,PESET-Evening-3,1.0,1.0,8.795419e+09,1.136955e-10,PESET,Evening,3,Hospital A,Hospital A-Evening-3
42226,315.1503@7.904,315.1503,7.904000,[M-H]-,Negative,Grab,PESET-Evening-3,1.0,1.0,8.795419e+09,1.136955e-10,PESET,Evening,3,Hospital A,Hospital A-Evening-3


In [4]:
#stacked barplot: Y is amount of unique compounds found, X is the unique samples, with positive and negative mode stacked
#mainly for sample qc
for tt in df['sampling_round'].unique():
    tmpdf = df[df['sampling_round'] == tt]
    
    # Group the data by 'sample name' and 'lcms_mode' and count the number of compounds in each group
    grouped = tmpdf.groupby(['Anonymized Sample ID', 'lcms_mode']).size().unstack(fill_value=0)
    
    print(grouped)

lcms_mode                             Negative  Positive
Anonymized Sample ID                                    
Hospital A-NA-1                        1048575       454
Hospital A-NA-2                        1048575       454
Hospital A-NA-3                        1048575       454
Hospital B-NA-1                        1048575       454
Hospital B-NA-2                        1048575       454
Hospital B-NA-3                        1048575       454
Hospital D-NA-1                        1048575       454
Hospital D-NA-2                        1048575       454
Hospital D-NA-3                        1048575       454
Hospital E-NA-1                        1048575       454
Hospital E-NA-2                        1048575       454
Hospital E-NA-3                        1048575       454
Hospital F-NA-1                        1048575       454
Hospital F-NA-2                        1048575       454
Hospital F-NA-3                        1048575       454
WW Collection Point City Center